ref: https://youtu.be/H9OmsD7F7p0

In [112]:
!pip install pythainlp
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [113]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
data = '/content/drive/MyDrive/Final_project/clean_data/combine_company.csv'

In [115]:
import pandas as pd
import numpy as np
from pythainlp.corpus import thai_stopwords
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [292]:
df = pd.read_csv(data)
frozen_stopword = thai_stopwords()

In [117]:
# Clean stop word and special charater
import string
from pythainlp.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stopword = stopwords.words('english')
thai_stopword = list(frozen_stopword)

def clean_tokenize(rec_text:str) -> list:
  clean_token = rec_text
  for pair in (('\n', ''), ('\t', ''), ('!', '')):
        clean_token =clean_token.replace(*pair)
  
  temp_tokenize = word_tokenize(clean_token, None, 'newmm', False)
  return temp_tokenize

def clean_stopword(word:list):
  temp_thai = []
  temp_eng = []
  for i in word:
    if i not in thai_stopword:
      temp_thai.append(i)
  
  for j in temp_thai:
    if j not in english_stopword:
      temp_eng.append(j)
    
  return temp_eng

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [203]:
# List company detail
"""
  ตัดรายละเอียดธุรกิจเป็นคำๆ แล้วใส่ในตัวแปร list_company_detail
"""

list_company_detail = []
for i in range(len(df)):
  detail = df.iloc[i]['รายละเอียดธุรกิจ'].lower()
  a = detail.translate(str.maketrans('','', string.punctuation))
  b = a.translate(str.maketrans('','', string.digits))
  temp_comp = clean_tokenize(b)
  stopword_cleaned = clean_stopword(temp_comp)
  list_company_detail.append(stopword_cleaned)

# หาค่า TF-IDF

In [204]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [205]:
def identity_fun(text):
    return text

In [293]:
tfidf_vectorizer = TfidfVectorizer(
    stop_words='english',
    analyzer='word',
    tokenizer=identity_fun,
    preprocessor=identity_fun,
    token_pattern=None
)

X = tfidf_vectorizer.fit_transform(list_company_detail)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn(


# KMeans clustering

In [294]:
from sklearn.cluster import KMeans
k = 10
km = KMeans(n_clusters=k)
km.fit(X)
centroid = km.cluster_centers_
cluster = km.labels_
df['cluster'] = cluster

In [ ]:
import random
temp_cluster = []
for i in range(len(df)):
  ind = random.randint(0, len(df)-1)
  if df.iloc[ind]['cluster'] == 8 and len(temp_cluster) < 10:
    temp_cluster.append([df.iloc[ind]['ชื่อย่อสถานประกอบการ'], df.iloc[ind]['รายละเอียดธุรกิจ']])


for k in temp_cluster:
  print(k[0])
  print(k[1])
  print()

In [295]:
cluster_map = {
    0: "ERP Solution / Accounting",
    1: "Designer / Multimedia",
    2: "IT / Infrastructure",
    3: "Hardware / Cloud",
    4: "Software Development",
    5: "Biometrics",
    6: "*AI / IT / VR / Digital solution",
    7: "Website",
    8: "Online Platform",
    9: "Digital Marketing / SEO"
}

# apply mapping
df['cluster'] = df['cluster'].map(cluster_map)

In [299]:
import random
for i in range(5):
  ind_sample = random.randint(0, len(df))
  print(f'{df.iloc[ind_sample]["ชื่อย่อสถานประกอบการ"]}')
  print(df.iloc[ind_sample]['รายละเอียดธุรกิจ'])
  print(f'cluster label: {cluster[ind_sample]}')
  print(f"Category: {df.iloc[ind_sample]['cluster']}")
  print()

FD
บริการ ออกแบบ พัฒนา ระบบเทคโนโลยีสารสนเทศ (Web & Mobile Development, Cloud Solution)
cluster label: 2
Category: IT / Infrastructure

MYCLOUD FULFILLMENT
บริการออกแบบและพัฒนาการให้บริการระบบคลังสินค้า และระบบจัดการออเดอร์สำหรับธุรกิจที่ขายสินค้าออนไลน์และขายหลายช่องทาง ผ่าน MYCLOUD fulfillment
cluster label: 8
Category: Online Platform

INNOSPRINT
บริการระบบขายตั๋วงานวิ่ง ปั่นจักรยาน และไตรกีฬา ทั้งระบบขายตั๋วออนไลน์, ระบบแจก Race Pack และระบบจับเวลา เจ้าของเว็บไซต์ RunLah.com
cluster label: 8
Category: Online Platform

ABSC
บริการออกแบบจัดหาพัฒนาระบบซอฟต์แวร์แบบครบวงจร (Oracle/Non Oracle application, financial, distributions, manufacturing (discrete & process manufacturing), CRM balanced scored card and business intelligence (BI) application implementation and consulting services)
cluster label: 7
Category: Website

BIG DATA
บริการพัฒนา ดูแลระบบซอฟต์แวร์ฮาร์ดแวร์ และบำรุงรักษาระบบการสอบสวนสืบสวนและพิสูจน์หลักฐาน (CCTV Solution, Business Process Management Software, Image Processing 

In [301]:
import collections
counter = collections.Counter(df['cluster'])
counter

Counter({'Designer / Multimedia': 443,
         'Online Platform': 181,
         'Biometrics': 94,
         'IT / Infrastructure': 125,
         'Software Development': 321,
         'ERP Solution / Accounting': 126,
         'Website': 167,
         'Digital Marketing / SEO': 81,
         '*AI / IT / VR / Digital solution': 77,
         'Hardware / Cloud': 28})